Starting with creating a CLI version of my program.

Starting off with creating a simple version of my program that can run with csv files. Does not take into account draft position or how many players your team can fit.

In [37]:
pip install sleeper-py
# found this resource on the internet when looking for sleeper api python on accident, decided to try it out

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# import packages
import pandas as pd
import requests
from pprint import pprint
import json
import csv

In [3]:
# constants
API = "https://api.sleeper.app/v1/players/nfl"

Starting with actual code

In [4]:
# get response from api request
response = requests.get(API)
display(response)
print(response.url)

<Response [200]>

https://api.sleeper.app/v1/players/nfl


In [5]:
def draft(player_name, player_dict, user_draft):
    '''finds player and add to list of user's players
    params: player_name (string)
            player_dict (dict)
            user_draft (list)
    returns: user_draft
    '''
    found_player = False
    for key in player_dict: # looking for player by iterating through dictionary
        player = player_dict[key]
        full_name = (player['first_name'] + " " + player['last_name']).lower()
        if full_name == player_name.lower(): # player is only appended if name in dictionary is same
            user_draft.append(player)
            print(f"you have drafted {player_name}")
            found_player = True

    if found_player == False: # returns false if player not found
        print(f"player {player_name} not found")
        return False
    else:
        return user_draft

In [6]:
def predicted_spots(position, league_size, team_size):
    '''def predicted_spots returns all predicted draft spots the user will draft at
    
    params: position (string): position that player's first pick is
            league_size (string): number of people in the league
            team_size (string): number of players on each team
    '''
    draft_order = [] # list for all picks
    current_pick = int(position) # starting at the first pick
    
    for round_number in range(int(team_size)):
        if round_number % 2 == 0: # regular order for odd rounds
            current_round = list(range(1, int(league_size) + 1))
        else: # reverse order for even rounds (snake draft)
            current_round = list(range(int(league_size), 0, -1))
        
        # calculate total pick number
        draft_order.append(current_pick + current_round[int(position) - 1] - 1)
        current_pick += int(league_size)
    
    return draft_order

In [37]:
def predicted_draft(league_type, draft_order):
    predicted = []

    if league_type == "standard":
        for spot in draft_order:
            spot_str = str(spot)
            for player in standard:
                if player['Rank'] == spot_str:
                    predicted.append(player['Player'])
    elif league_type == "full ppr":
        for spot in draft_order:
            spot_str = str(spot)
            for player in full_ppr:
                if player['Rank'] == spot_str:
                    predicted.append(player['Player'])
    else:
        for spot in draft_order:
            spot_str = str(spot)
            for player in half_ppr:
                if player['Rank'] == spot_str:
                    predicted.append(player['Player'])

    return predicted

In [38]:
# testing predicted_spots
print(predicted_spots("1", "12", "15"))

[1, 24, 25, 48, 49, 72, 73, 96, 97, 120, 121, 144, 145, 168, 169]


In [39]:
# testing predicted_draft
test_predicted_spots = predicted_spots("1", "12", "15")
print(predicted_draft("standard", test_predicted_spots))

['Christian McCaffrey', 'Josh Jacobs', 'Travis Kelce', 'Trey McBride', 'Mark Andrews', 'Rome Odunze', 'Zamir White', 'Devin Singletary', 'Brock Purdy', 'Curtis Samuel', 'Romeo Doubs', 'Tyler Allgeier', 'Gabe Davis', 'Isaiah Likely', 'Quentin Johnston']


In [ ]:
def calculate_grade(user_draft, draft_positions, league_type):
    '''
    def calculate_grade calculates draft grade by predicted ranking

    params: user_draft (list): list of players user has drafted
            draft_positions (list): list of positions user is drafting in
    returns: grade (string): letter grade assigned to value, prints predicted score and your score as well
    '''

    predicted_score = 0 # predicted score is based on player ranking from previous seasons, by adding all the picks we should get the predicted score as the predicted draft will be the player rated at that draft pick (i.e. the 72nd pick will be the 72nd best player)
    for round in draft_positions:
        predicted_score += round
    
    print(f"your predicted score is: {predicted_score}")

    # if statements for each type of scoring
    # in each if statement, use player_name from user_draft players to find player ranking in separate scoring system
    # for key in (standard, etc.)
    # add those rankings together to create predicted score
    # probably need to convert type from string to int and then add everything up

    score = 0

    if league_type == "standard":
        for player in user_draft:
            full_name = (player['first_name'] + " " + player['last_name']).lower()
            

    for player in user_draft:
        score += player['search_rank']

    print(f"your score is: {score}") # printing both predicted and real score to let user see a little on how it was graded

    

    print("your draft grade is: ")
    if score <= predicted_score - 75:
        return "S"
    if score <= predicted_score - 50:
        return "A"
    if score <= predicted_score - 25:
        return "B"
    if score <= predicted_score or score <= predicted_score + 25:
        return "C"
    if score <= predicted_score + 50:
        return "D"
    else:
        return "F"

In [10]:
def read_players(filename):
    '''def read_players reads in players into json file

    params: filename (string): filename to read
    returns: player_dict (dictionary): dictionary with all players
    '''
    with open(filename , "r") as fo:
        player_dict = json.load(fo) 
    return player_dict

In [23]:
def csv_to_json(csv_filename, json_filename):
    # Read the CSV file
    with open(csv_filename, mode='r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
            
        # Convert rows to a list of dictionaries
        data = [row for row in csv_reader]

    # Write data to a JSON file
    with open(json_filename, mode='w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=4)

    print(f"JSON file has been created at: {json_filename}")
    return json_filename

In [34]:
user_draft = []
player_dict = read_players("players.json")
standard_json = csv_to_json("beatadp_standard_combined__sleeper_rankings.csv", "standard.json")
half_ppr_json = csv_to_json("beatadp_halfppr_combined__sleeper_rankings.csv", "half ppr.json")
full_ppr_json = csv_to_json("beatadp_ppr_combined__sleeper_rankings.csv", "full ppr.json")
standard = read_players("standard.json")
half_ppr = read_players("half ppr.json")
full_ppr = read_players("full ppr.json")

JSON file has been created at: standard.json
JSON file has been created at: half ppr.json
JSON file has been created at: full ppr.json


In [ ]:
user_draft = []
player_dict = read_players("players.json")
standard = csv_to_json("beatadp_standard_combined__sleeper_rankings.csv", "standard")
half_ppr = csv_to_json("beatadp_halfppr_combined__sleeper_rankings.csv", "half ppr")
full_ppr = csv_to_json("beatadp_ppr_combined__sleeper_rankings.csv", "full ppr")

league_size = input("how many people are in your league? (input int, not string)")
team_size = input("how many players are on each team? (input int, not string)")
position = input("what number pick is your first pick? (input int, not string)")
league_type = input("please input standard, full ppr or half ppr for your league type:")

draft_positions = predicted_spots(position, league_size, team_size)

x = 0 # counter for how many picks
while x < int(team_size): # user picks team
    player_name = input("you are currently picking at pick " + str(draft_positions[x]) + ". please input your player's full name!")

    drafted = draft(player_name, player_dict, user_draft) # originally this was in the if statement but it was printing twice so i fixed by doing this

    if drafted == False:
        print("try again!")
    else:
        user_draft = drafted
        x += 1

print(calculate_grade(user_draft, draft_positions))

Conversion successful! JSON file saved at: standard
Conversion successful! JSON file saved at: half ppr
Conversion successful! JSON file saved at: full ppr


ValueError: invalid literal for int() with base 10: ''